In [1]:
import pandas as pd
import os
!git clone https://github.com/Benjaxmen/pm-prediction
!cd pm-prediction/

Cloning into 'pm-prediction'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 402 (delta 3), reused 9 (delta 2), pack-reused 390
Receiving objects: 100% (402/402), 67.36 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (342/342), done.


In [2]:
def find_csv_files(root_folder):
    csv_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
          if file.endswith('.csv'):
              csv_files.append(os.path.join(root, file))
    return csv_files


In [3]:
stations=[]
for each in files:
  if os.path.basename(os.path.dirname(each)) not in stations:
    stations.append(os.path.basename(os.path.dirname(each)))
print(stations)

['hualqui', 'liceo polivalente', 'laja', 'coronel sur', 'nueva libertad', 'lagunillas enel', 'escuadron enel', 'lota rural', 'colicheu', 'lautaro', 'club de empleados', 'coronel norte', 'lota urbana', 'kingston college', 'consultorio san vicente']


In [20]:
def agregar_ceros(valor):
    if len(str(valor)) < 6:
      if len(str(valor))<=4 or len(str(valor))>6:
        return float('NaN')
      return "0" + str(valor)
    else:
        return valor
def get_file_name(file):
    base_name = os.path.basename(file)
    name_without_ext = os.path.splitext(base_name)[0]
    if name_without_ext.lower() == 'mp25':
        name_without_ext = 'mp2.5'
    return name_without_ext

def process_csv_files(root_folder):
    all_data = {}  # Diccionario para guardar los datos de todas las carpetas
    for root, dirs, files in os.walk(root_folder):
        folder_data = {}  # Diccionario para guardar los DataFrames de cada carpeta
        for file in files:
            if file.endswith('.csv'):
                csv_file_path = os.path.join(root, file)
                station = os.path.basename(os.path.dirname(csv_file_path))
                #comment this if statement if you are unifying super site and escuela e10
                if station == "super site - escuela e10" or station == "escuela e10" or station == "supersite":
                  continue
                df = pd.read_csv(csv_file_path, sep=';')
                file_name = get_file_name(csv_file_path)
                # Apply data cleaning operations
                if 'Unnamed: 2' in df.columns and 'Unnamed: 3' in df.columns:
                    df = df.dropna(subset=['Unnamed: 2', 'Unnamed: 3'], how='all')
                    df[file_name] = df['Unnamed: 2'].combine_first(df['Unnamed: 3'])
                    df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'HORA (HHMM)'], axis=1)
                elif 'Registros preliminares' in df.columns and 'Registros no validados' in df.columns and 'Registros validados' in df.columns:
                    df.drop('Unnamed: 5', axis=1, inplace=True)
                    df.dropna(subset=['Registros preliminares', 'Registros no validados', 'Registros validados'], how='all', inplace=True)
                    df[file_name] = df['Registros preliminares'].combine_first(df['Registros no validados'])
                    df[file_name] = df['Registros validados'].combine_first(df[file_name])
                    df.drop(['Registros preliminares', 'Registros no validados', 'Registros validados', 'HORA (HHMM)'], axis=1, inplace=True)
                if df[file_name].dtype == 'object':
                  df[file_name] = df[file_name].str.replace(',', '.').astype(float)
                else:
                  print(f"La columna {file_name} no es de tipo string.")
                df['FECHA (YYMMDD)'] = df['FECHA (YYMMDD)'].apply(agregar_ceros)
                df['FECHA (YYMMDD)'].dropna(inplace=True)

                df['FECHA (YYMMDD)'] = pd.to_datetime(df['FECHA (YYMMDD)'], format='%y%m%d')

                # Agregar el nombre de la estación como una columna
                df['station'] = station

                # Si la columna ya existe en el diccionario de la carpeta, combina los datos
                if file_name in folder_data:
                    folder_data[file_name] = pd.concat([folder_data[file_name], df], ignore_index=True)
                else:
                    folder_data[file_name] = df

        # Añadir los datos de la carpeta al diccionario general
        all_data[root.split("/")[-1].split(".")[0]] = folder_data

    return all_data

def consolidate_station_data(all_data):
    consolidated_data = {}
    for station, folder_data in all_data.items():
        station_dfs = list(folder_data.values())
        if station_dfs:
            # Concatenar los DataFrames contenidos dentro de cada entrada del diccionario
            merged_df = pd.concat(station_dfs, ignore_index=True)
            merged_df['FECHA (YYMMDD)'] = pd.to_datetime(merged_df['FECHA (YYMMDD)'])
            merged_df = merged_df.groupby('FECHA (YYMMDD)').first().reset_index()
            consolidated_data[station] = merged_df
    return consolidated_data

def concatenate_all_data(all_data):
    concatenated_dfs = []
    for folder_data in all_data.values():
        for df in folder_data.values():
            concatenated_dfs.append(df)
    concatenated_df = pd.concat(concatenated_dfs, ignore_index=True)
    return concatenated_df




In [6]:
files=find_csv_files("/content/pm-prediction/biobio")
all_data = process_csv_files("/content/pm-prediction/biobio")
consolidated_data = consolidate_station_data(all_data)
all_dataframes = pd.concat(consolidated_data.values(), ignore_index=True)
all_dataframes.to_csv("biobio.csv",index=False)

La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.


In [7]:
files=find_csv_files("/content/pm-prediction/metropolitana")
all_data = process_csv_files("/content/pm-prediction/metropolitana")
consolidated_data = consolidate_station_data(all_data)
all_dataframes = pd.concat(consolidated_data.values(), ignore_index=True)
all_dataframes.to_csv("metropolitana.csv",index=False)

<ipython-input-4-d9cbd49f1cb0>:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep=';')
<ipython-input-4-d9cbd49f1cb0>:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep=';')
<ipython-input-4-d9cbd49f1cb0>:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep=';')
<ipython-input-4-d9cbd49f1cb0>:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep=';')


In [19]:
#Esta es la misma estación, sólo que cambió de nombre
!cd /content/pm-prediction/antofagasta
all_data = process_csv_files("/content/pm-prediction/antofagasta/super site - escuela e10/escuela e10")
consolidated_data = consolidate_station_data(all_data)
all_dataframes = pd.concat(consolidated_data.values(), ignore_index=True)
all_dataframes.to_csv("escuela e10.csv",index=False)
all_data = process_csv_files("/content/pm-prediction/antofagasta/super site - escuela e10/supersite")
consolidated_data = consolidate_station_data(all_data)
all_dataframes = pd.concat(consolidated_data.values(), ignore_index=True)
all_dataframes.to_csv("supersite.csv")
supersite = pd.read_csv("supersite.csv")
escuela = pd.read_csv("escuela e10.csv",index=False)

supersite = pd.concat([escuela, supersite], ignore_index=True)
supersite['station']="supersite-escuela e10"
supersite.to_csv("supersite-escuela e10.csv",index=False)
supersite.head(10)

La columna o3 no es de tipo string.
La columna mp10 no es de tipo string.
La columna mp2.5 no es de tipo string.
La columna hrel no es de tipo string.


,Unnamed: 0,FECHA (YYMMDD),no2,station,o3,nox,mp10,no,mp2.5,dirviento,co,tamb,so2,velviento,hrel
0,0,2001-01-03,NaN,supersite-escuela e10,NaN,NaN,37.7300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2001-01-05,NaN,supersite-escuela e10,NaN,NaN,39.0200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2001-01-08,NaN,supersite-escuela e10,NaN,NaN,66.0597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2001-01-10,NaN,supersite-escuela e10,NaN,NaN,57.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2001-01-15,NaN,supersite-escuela e10,NaN,NaN,50.2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2001-01-17,NaN,supersite-escuela e10,NaN,NaN,48.9200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2001-01-19,NaN,supersite-escuela e10,NaN,NaN,36.6299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2001-01-22,NaN,supersite-escuela e10,NaN,NaN,77.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2001-01-24,NaN,supersite-escuela e10,NaN,NaN,90.4600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2001-01-26,NaN,supersite-escuela e10,NaN,NaN,90.7699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
all_data = process_csv_files("/content/pm-prediction/antofagasta")
consolidated_data = consolidate_station_data(all_data)
all_dataframes = pd.concat(consolidated_data.values(), ignore_index=True)
supersite=pd.read_csv("supersite-escuela e10.csv",index=False)
all_dataframes = pd.concat([all_dataframes, supersite], ignore_index=True)
all_dataframes.to_csv("antofagasta.csv")

La columna mp2.5 no es de tipo string.


In [ ]:
antofagasta = pd.read_csv("antofagasta.csv")
antofagasta['region'] = 0
metropolitana = pd.read_csv("metropolitana.csv")
metropolitana['region'] = 1
biobio = pd.read_csv("biobio.csv")
biobio['region'] = 2
all_dataframes = pd.concat([antofagasta,metropolitana,biobio],ignore_index=True)
all_dataframes.to_csv("consolidado.csv",index=False)